# Trading Signal Generation

## Setup

In [ ]:
# -------------------- Core Libraries --------------------
import os
import re
import pandas as pd
import numpy as np
import torch

# -------------------- Text Processing --------------------
import spacy
from preprocessing import preprocess_text
from transformers import (
    AutoTokenizer,
    AutoModel,
)

# -------------------- Machine Learning --------------------
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
)

# -------------------- Visualization --------------------
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm  # Jupyter-native progress bars

In [ ]:
# Change working directory to the root of the project (go up one directory from notebooks to root)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.chdir(project_root)

## Preprocessing

In [ ]:
january = "data/processed/bdm/2024-01.csv"
with open(january, "r", encoding="utf-8") as file:
    df_jan = pd.read_csv(file)

february = "data/processed/bdm/2024-02.csv"
with open(february, "r", encoding="utf-8") as file:
    df_feb = pd.read_csv(file)

march = "data/processed/bdm/2024-03.csv"
with open(march, "r", encoding="utf-8") as file:
    df_march = pd.read_csv(file)

In [ ]:
# -------------------- Preprocessing --------------------
import re
import spacy

# Load spaCy Portuguese model once
nlp = spacy.load("pt_core_news_sm")

# Dictionary for acronym expansion
acronyms = {
    "Selic": "Sistema Especial de Liquidação e de Custódia",
    "PIB": "Produto Interno Bruto",
    "CDI": "Certificado de Depósito Interbancário",
    "LPRs": "Loan Prime Rates",
    "Ibovespa": "Índice Bovespa",
    "BB": "Banco do Brasil",
    "BC": "Banco Central",
    "FGTS": "Fundo de Garantia do Tempo de Serviço",
    "STF": "Supremo Tribunal Federal",
    "CPI": "Índice de Preços ao Consumidor",
    "MP": "Medida Provisória",
    "EUA": "Estados Unidos",
    "ONU": "Organização das Nações Unidas",
    "FGV": "Fundação Getúlio Vargas",
    "IBGE": "Instituto Brasileiro de Geografia e Estatística",
    "BNDES": "Banco Nacional de Desenvolvimento Econômico e Social",
    "IPCA": "Índice Nacional de Preços ao Consumidor Amplo",
    "DI": "Depósito Interfinanceiro",
    "IR": "Imposto de Renda",
    "OI": "Operadora Oi",
    "CV": "Câmara de Vereadores"
}

# Noisy acronyms to remove
noisy_acronyms = {"ROMI", "ENEVA", "LIGHT", "DA"}

def normalize_numbers(text):
    text = re.sub(r"R\$ ?([\d.,]+) bilhões", r"\1B", text)
    text = re.sub(r"R\$ ?([\d.,]+) milhões", r"\1M", text)
    text = re.sub(r"([\d.,]+) pp", r"\1%", text)
    text = text.replace(",", "")
    return text

def expand_acronyms(text, acronym_dict):
    for acronym, full_form in acronym_dict.items():
        text = re.sub(rf'\b{re.escape(acronym)}\b', full_form, text, flags=re.IGNORECASE)
    return text

def remove_noisy_acronyms(text, noisy_set):
    return re.sub(r'\b(?:' + '|'.join(noisy_set) + r')\b', '', text)

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

def preprocess_text(text):
    text = normalize_numbers(text)
    text = expand_acronyms(text, acronyms)
    text = remove_noisy_acronyms(text, noisy_acronyms)
    text = lemmatize_text(text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

In [ ]:
# Apply preprocessing to the dataset
df_jan['cleaned_article'] = df_jan['article'].apply(preprocess_text)
df_feb['cleaned_article'] = df_feb['article'].apply(preprocess_text)
df_march['cleaned_article'] = df_march['article'].apply(preprocess_text)

## Data Exploration

In [ ]:
print(df_jan['label'].value_counts())
sns.countplot(x='label', data=df_jan)
plt.title('Label Distribution in January Dataset')
plt.savefig("results/bert_embeddings_experiment_v1/figures/label_distribution_january.png")

In [ ]:
print(df_feb['label'].value_counts())
sns.countplot(x='label', data=df_feb)
plt.title('Label Distribution in February Dataset')
plt.savefig("results/bert_embeddings_experiment_v1/figures/label_distribution_february.png")

In [ ]:
print(df_march['label'].value_counts())
sns.countplot(x='label', data=df_march)
plt.title('Label Distribution in March Dataset')
plt.savefig("results/bert_embeddings_experiment_v1/figures/label_distribution_march.png")

## Generate BERT Word Embeddings

In [ ]:
# Load BERTimbau tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = AutoModel.from_pretrained("neuralmind/bert-base-portuguese-cased")

In [ ]:
def get_bert_embedding(text, tokenizer, model):
    # tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # pass inputs through model
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract [CLS] token embedding (shape: [batch_size, hidden_size])
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # [CLS] token is the first token
    return cls_embedding.squeeze(0).numpy()  # convert to NumPy array

In [ ]:
df_jan['embedding'] = df_jan['cleaned_article'].apply(lambda x: np.array(get_bert_embedding(x, tokenizer, model)))
df_feb['embedding'] = df_feb['cleaned_article'].apply(lambda x: np.array(get_bert_embedding(x, tokenizer, model)))
df_march['embedding'] = df_march['cleaned_article'].apply(lambda x: np.array(get_bert_embedding(x, tokenizer, model)))

## Logistic Regression Classification
- train on January/February corpus
- test on March corpus
- repeated for both with and without neutral classifications (0)

In [ ]:
# Combine January and February data for training
df_train_multi = pd.concat([df_jan, df_feb])
X_train_multi = np.vstack(df_train_multi['embedding'].values)
y_train_multi = df_train_multi['label']

# multi classification and test on march
X_test_multi = np.vstack(df_march['embedding'].values)
y_test_multi = df_march['label']

# binary Classification and test on march
df_train_binary = df_train_multi[df_train_multi['label'] != 0]
df_march_binary = df_march[df_march['label'] != 0]

X_train_binary = np.vstack(df_train_binary['embedding'].values)
y_train_binary = df_train_binary['label']

X_test_binary = np.vstack(df_march_binary['embedding'].values)
y_test_binary = df_march_binary['label']

In [ ]:
# Train multi-class classifier
multi_clf = LogisticRegression(max_iter=1000)
multi_clf.fit(X_train_multi, y_train_multi)

# Predict and evaluate
y_pred_multi = multi_clf.predict(X_test_multi)
print("Multi-Class Report:\n", classification_report(y_test_multi, y_pred_multi))

In [ ]:
# Train binary classifier
binary_clf = LogisticRegression(max_iter=1000)
binary_clf.fit(X_train_binary, y_train_binary)

# Predict and evaluate
y_pred_binary = binary_clf.predict(X_test_binary)
print("Binary Classification Report:\n", classification_report(y_test_binary, y_pred_binary))

In [ ]:
results_dir = "results/bert_embeddings_experiment_v1/metrics"
os.makedirs(results_dir, exist_ok=True)

cm_multi = confusion_matrix(y_test_multi, y_pred_multi)
class_report = classification_report(y_test_multi, y_pred_multi, target_names=["-1", "0", "1"])

report_path = os.path.join(results_dir, "classification_report_multi.txt")
with open(report_path, "w") as f:
    f.write("Classification Report:\n")
    f.write(class_report)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_multi, annot=True, fmt="d", cmap="Blues", 
            xticklabels=["-1", "0", "1"], yticklabels=["-1", "0", "1"])
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Multi-Class Confusion Matrix")

conf_matrix_path = os.path.join(results_dir, "confusion_matrix_multi.png")
plt.savefig(conf_matrix_path)
plt.close()

print(f"Classification report saved to: {report_path}")
print(f"Confusion matrix saved to: {conf_matrix_path}")

In [ ]:
cm_binary = confusion_matrix(y_test_binary, y_pred_binary)
class_report_binary = classification_report(y_test_binary, y_pred_binary, target_names=["-1", "1"])

report_path_binary = os.path.join(results_dir, "classification_report_binary.txt")
with open(report_path_binary, "w") as f:
    f.write("Classification Report (Binary):\n")
    f.write(class_report_binary)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_binary, annot=True, fmt="d", cmap="Greens", 
            xticklabels=["-1", "1"], yticklabels=["-1", "1"])
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Binary Confusion Matrix")

conf_matrix_path_binary = os.path.join(results_dir, "confusion_matrix_binary.png")
plt.savefig(conf_matrix_path_binary)
plt.close()

print(f"Binary classification report saved to: {report_path_binary}")
print(f"Binary confusion matrix saved to: {conf_matrix_path_binary}")

## Miss-classification Results

In [ ]:
# Add multi-class predictions to df_march
df_march['classification_multi'] = y_pred_multi

# Add binary predictions to df_march_binary
df_march_binary['classification_binary'] = y_pred_binary

# Create a DataFrame containing the articles, labels, and classifications
df_combined_classifications = df_march[["cleaned_article", 'label', 'classification_multi']].copy()
if not df_march_binary.empty:
    # Add binary classifications for binary-labeled articles
    df_combined_classifications['classification_binary'] = np.nan
    df_combined_classifications.loc[
        df_combined_classifications.index.isin(df_march_binary.index), 'classification_binary'
    ] = df_march_binary['classification_binary']

# Save the results to a CSV file
output_path = "results/bert_embeddings_experiment_v1/metrics/articles-with-misclassified-multi.csv"
df_combined_classifications.to_csv(output_path, index=False)

# Display the resulting DataFrame for review
df_combined_classifications

print(f"Articles with classifications have been saved to {output_path}.")


In [ ]:
# filter for multi-class misclassifications
df_misclassified_multi = df_march[df_march['classification_multi'] != df_march['label']][
    ['cleaned_article', 'label', 'classification_multi']
]

df_misclassified_multi['misclassification_type'] = df_misclassified_multi.apply(
    lambda row: f"{row['label']} misclassified as {row['classification_multi']}", axis=1
)

# totals for each misclassification type (multi-class)
multi_counts = df_misclassified_multi['misclassification_type'].value_counts()
misclassified_multi_path = "results/bert_embeddings_experiment_v1/metrics/misclassified-multi.csv"
df_misclassified_multi.to_csv(misclassified_multi_path, index=False)

print("Multi-Class Misclassification Totals:")
print(multi_counts)

In [ ]:
# filter for binary misclassifications
df_misclassified_binary = df_march_binary[df_march_binary['classification_binary'] != df_march_binary['label']][
    ['cleaned_article', 'label', 'classification_binary']
]

df_misclassified_binary['misclassification_type'] = df_misclassified_binary.apply(
    lambda row: f"{row['label']} misclassified as {row['classification_binary']}", axis=1
)

# totals for each misclassification type (binary)
binary_counts = df_misclassified_binary['misclassification_type'].value_counts()
misclassified_binary_path = "results/bert_embeddings_experiment_v1/metrics/misclassified-binary.csv"
df_misclassified_binary.to_csv(misclassified_binary_path, index=False)

print("Binary Misclassification Totals:")
print(binary_counts)